# 智能库存管理 Agent：需求预测与 Milvus 语义搜索

**目标**：本 Notebook 旨在演示智能库存管理 Agent 的核心功能，包括：
1.  **需求预测**：基于历史数据，使用简单的机器学习模型预测未来汽车零件的需求。
2.  **Milvus 集成**：将零件信息（特别是文本描述）向量化，并存入 Milvus 向量数据库，以实现快速的语义搜索。

**环境**：此 Notebook 可在 Google Colab 中直接运行。您只需上传此 `.ipynb` 文件，或将代码复制到新的 Colab Notebook 中即可。

**注意**：部分代码块（如 Milvus 连接）需要您提供自己的 Milvus 实例 URI 和 Token。示例代码将使用占位符。

## 1. 环境设置

首先，我们安装并导入所有必需的 Python 库。在 Google Colab 中，您需要取消注释并运行以下命令来安装 `pymilvus` 和 `sentence-transformers`。

In [ ]:
# 在 Google Colab 中运行时，请取消以下代码的注释
# !pip install pandas numpy scikit-learn matplotlib pymilvus sentence-transformers

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# 模拟 Milvus 和 SentenceTransformer 的功能，以便在没有这些库的情况下运行
try:
    from pymilvus import MilvusClient
    from sentence_transformers import SentenceTransformer
except ImportError:
    print("Pymilvus or Sentence-Transformers not installed. Using mock objects for demonstration.")
    # 创建一个模拟的 MilvusClient 类
    class MockMilvusClient:
        def __init__(self, uri, token):
            print(f"Mock Milvus connection to {uri} initialized.")
        def create_collection(self, collection_name, dimension):
            print(f"Mock collection '{collection_name}' with dimension {dimension} created.")
        def insert(self, collection_name, data):
            print(f"Mock insert of {len(data)} vectors into '{collection_name}'.")
            return {"insert_count": len(data)}
        def search(self, collection_name, data, limit):
            print(f"Mock search in '{collection_name}' completed.")
            return [[{"id": 1, "distance": 0.9, "entity": {'part_name': 'Brake Pad'}}]]

    # 创建一个模拟的 SentenceTransformer 类
    class MockSentenceTransformer:
        def __init__(self, model_name):
            print(f"Mock SentenceTransformer with model '{model_name}' initialized.")
        def encode(self, texts):
            print(f"Mock encoding of {len(texts)} texts.")
            return np.random.rand(len(texts), 384) # 返回一个随机的向量

    MilvusClient = MockMilvusClient
    SentenceTransformer = MockSentenceTransformer

## 2. 模拟生成汽车零件库存数据

为了进行演示，我们创建一个包含汽车零件信息的模拟数据集。该数据集包括零件 ID、名称、描述、当前库存和过去12个月的历史需求。

In [ ]:
def create_mock_data(num_parts=50):
    data = {
        'part_id': [i for i in range(1, num_parts + 1)],
        'part_name': [f'Part-{chr(65 + i % 26)}{i}' for i in range(num_parts)],
        'part_description': [
            'High-performance brake pad for sedans', 'Standard oil filter for gasoline engines',
            'All-weather synthetic tire', 'LED headlight bulb, 6000K bright white',
            'Engine air filter, improves airflow'
        ] * (num_parts // 5),
        'stock_level': np.random.randint(50, 500, size=num_parts),
        'supplier': [f'Supplier-{chr(88 + i % 3)}' for i in range(num_parts)],
        'price': np.round(np.random.uniform(20.0, 500.0, size=num_parts), 2)
    }
    
    # 生成过去12个月的需求数据
    for i in range(1, 13):
        data[f'demand_month_{i}'] = np.random.randint(10, 100, size=num_parts) + i * 5 # 添加趋势
        
    df = pd.DataFrame(data)
    return df

inventory_df = create_mock_data()

print("Generated Inventory Data (First 5 rows):")
inventory_df.head()

## 3. 需求预测

我们将使用一个简单的线性回归模型来预测下一个月（第13个月）的零件需求。我们将使用过去12个月的数据作为训练集。

In [ ]:
def forecast_demand(df):
    demand_cols = [f'demand_month_{i}' for i in range(1, 13)]
    X = np.arange(1, 13).reshape(-1, 1) # 月份 (1-12)
    
    predictions = []
    for index, row in df.iterrows():
        y = row[demand_cols].values # 历史需求
        
        model = LinearRegression()
        model.fit(X, y)
        
        # 预测第13个月的需求
        predicted_demand = model.predict(np.array([[13]]))[0]
        predictions.append(int(predicted_demand))
        
    df['predicted_demand_month_13'] = predictions
    return df

inventory_df = forecast_demand(inventory_df)

print("Inventory Data with Demand Forecast (First 5 rows):")
inventory_df[['part_id', 'part_name', 'predicted_demand_month_13']].head()

### 可视化预测结果

为了更直观地理解预测效果，我们随机选择一个零件并绘制其历史需求和预测需求。

In [ ]:
def plot_forecast(df, part_id):
    part_data = df[df['part_id'] == part_id].iloc[0]
    
    history_months = np.arange(1, 13)
    history_demand = part_data[[f'demand_month_{i}' for i in range(1, 13)]].values
    
    forecast_month = 13
    forecast_demand = part_data['predicted_demand_month_13']
    
    plt.figure(figsize=(10, 6))
    plt.plot(history_months, history_demand, 'o-', label='Historical Demand')
    plt.plot(forecast_month, forecast_demand, 'ro', label='Forecasted Demand')
    plt.title(f'Demand Forecast for {part_data['part_name']}')
    plt.xlabel('Month')
    plt.ylabel('Demand')
    plt.xticks(np.arange(1, 14))
    plt.legend()
    plt.grid(True)
    plt.show()

# 随机选择一个零件进行可视化
sample_part_id = np.random.randint(1, len(inventory_df) + 1)
plot_forecast(inventory_df, sample_part_id)

## 4. Milvus 集成与语义搜索

现在，我们将演示如何将零件的文本描述转换为向量，并将其存储在 Milvus 中以进行语义搜索。这对于查找功能相似但名称不同的零件非常有用。

### 步骤 1: 文本向量化
我们使用 `sentence-transformers` 模型将零件描述转换为向量嵌入。这些向量能够捕捉文本的语义信息。

In [ ]:
# 加载预训练的嵌入模型
# 在实际应用中，请选择适合您语言和领域的模型
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# 为每个零件描述生成向量嵌入
descriptions = inventory_df['part_description'].tolist()
embeddings = embedding_model.encode(descriptions)

inventory_df['embedding'] = list(embeddings)

print(f"Generated {len(embeddings)} embeddings.")
print("Embedding dimension:", embeddings[0].shape)

### 步骤 2: 连接到 Milvus 并存储数据

接下来，我们连接到 Milvus 实例，创建一个集合，并将零件数据（包括其向量嵌入）插入到集合中。

**注意**: 请将 `YOUR_MILVUS_URI` 和 `YOUR_MILVUS_TOKEN` 替换为您自己的 Milvus 实例信息。

In [ ]:
# Milvus 连接信息 (请替换为您的真实信息)
MILVUS_URI = "http://milvus-server:19530" # 例如: "http://localhost:19530"
MILVUS_TOKEN = "username:password"      # 例如: "root:Milvus"
COLLECTION_NAME = "auto_inventory"

# 初始化 Milvus 客户端
milvus_client = MilvusClient(uri=MILVUS_URI, token=MILVUS_TOKEN)

# 创建一个集合来存储汽车零件数据
# 如果集合已存在，可以先删除或直接使用
try:
    milvus_client.drop_collection(collection_name=COLLECTION_NAME)
except: # noqa E722
    pass

milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=embeddings[0].shape[0] # 向量维度
)

print(f"Collection '{COLLECTION_NAME}' created successfully.")

In [ ]:
# 准备要插入 Milvus 的数据
data_to_insert = [
    {"vector": row['embedding'], "part_name": row['part_name'], "part_id": row['part_id']}
    for index, row in inventory_df.iterrows()
]

# 插入数据
res = milvus_client.insert(collection_name=COLLECTION_NAME, data=data_to_insert)
print(f"Inserted {res['insert_count']} entities into Milvus.")

### 步骤 3: 执行语义搜索

现在数据已经存储在 Milvus 中，我们可以执行语义搜索。例如，我们可以搜索与“适用于轿车的轮胎”功能相似的零件。

In [ ]:
# 定义一个搜索查询
search_query = "tires for a sedan car"

# 将查询文本转换为向量
query_vector = embedding_model.encode([search_query])[0]

# 在 Milvus 中执行向量搜索
search_results = milvus_client.search(
    collection_name=COLLECTION_NAME,
    data=[query_vector],
    limit=3, # 返回最相似的3个结果
    output_fields=["part_name", "part_id"] # 指定要返回的字段
)

print(f"Search results for query: '{search_query}'")
for result in search_results[0]:
    print(f"  - Part ID: {result['entity']['part_id']}, Name: {result['entity']['part_name']}, Distance: {result['distance']:.4f}")

## 5. 结论与后续步骤

本 Notebook 演示了智能库存管理 Agent 的两个核心功能：
1.  **需求预测**：通过简单的线性回归模型，我们成功地根据历史数据对未来需求进行了预测。在实际应用中，可以替换为更复杂的模型（如 LSTM、ARIMA）以提高准确性。
2.  **语义搜索**：通过将零件描述向量化并存储在 Milvus 中，我们实现了强大的语义搜索功能，这有助于仓库管理员快速找到相似或可替代的零件。

**后续步骤**：
- **模型优化**：使用更先进的时间序列预测模型，并进行超参数调优。
- **Agent 集成**：将预测和搜索逻辑封装到 Agent 的 `inventory_service` 和 `tools` 中。
- **自动化流程**：创建一个 `run_agent.py` 脚本，定期运行预测任务，并在库存低于安全阈值时自动触发补货建议。
- **API 开发**：在 `api/` 目录下开发 RESTful API，供仓库管理系统或前端界面调用。